In [3]:
# Set openai key
# Please gen and add `OPENAI_API_KEY` in the `.env` file in the current directory
# https://platform.openai.com/account/api-keys
import os

from dotenv import load_dotenv

load_dotenv()

True

In [17]:
LLM_MODEL = "gpt-3.5-turbo"

### Legacy Chains

#### LLM chain

In [72]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.9, model=LLM_MODEL)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Who is {person}?"
)

chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
chain.run("Angus Young")

In [ ]:
chain.run("Freddie Mercury")

#### Sequential chain

In [73]:
# prompt template 1: 
first_prompt = ChatPromptTemplate.from_template(
    "Who is {person}?"
)
# chain 1: input -> person and output -> person_info
first_chain = LLMChain(
    llm=llm,
    prompt=first_prompt, 
    output_key="person_info",
)

In [74]:
# prompt template 2: 
second_prompt = ChatPromptTemplate.from_template(
    "Squeeze the description of the person into one sentence with maximum 20 words: {person_info}?"
)
# chain 2: input -> person_info and output -> short_info
second_chain = LLMChain(
    llm=llm,
    prompt=second_prompt, 
    output_key="short_info",
)

In [75]:
# prompt template 3: 
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following sentence into the Spanish language: {short_info}?"
)
# chain 3: input -> short_info and output -> translated_info
third_chain = LLMChain(
    llm=llm,
    prompt=third_prompt,
    output_key="translated_info",
)

In [76]:
from langchain.chains import SequentialChain

sequential_chain = SequentialChain(
    chains=[first_chain, second_chain, third_chain],
    input_variables=["person"],
    output_variables=["person_info", "short_info", "translated_info"],
    verbose=True,
)

sequential_chain("Angus Young")



> Entering new SequentialChain chain...

> Finished chain.


{'person': 'Angus Young',
 'person_info': "Angus Young is a Scottish-born Australian musician and songwriter. He is best known as the co-founder, lead guitarist, and songwriter of the rock band AC/DC. Young is known for his energetic performances, distinctive stage presence, and his iconic schoolboy outfit, which has become a trademark of the band. He has been a key member of AC/DC since its formation in 1973 and has contributed to the band's massive success over the years.",
 'short_info': 'Angus Young, the Scottish-born Australian musician, is a co-founder, lead guitarist, and songwriter of AC/DC, known for his energetic performances and iconic schoolboy outfit.',
 'translated_info': 'Angus Young, el músico escocés-australiano nacido en Escocia, es uno de los cofundadores, guitarrista principal y compositor de AC/DC, conocido por sus enérgicas actuaciones y su icónico atuendo de colegial.'}

### New API Chains

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model=LLM_MODEL)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "When {person} was born?"
)

chain = prompt | llm

In [ ]:
response = chain.invoke({"person": "Angus Young"})
response.content

In [ ]:
response = chain.invoke({"person": "Freddie Mercury"})
response.content

#### Output parsers

In [ ]:
from langchain.output_parsers import DatetimeOutputParser

datetim_output_parser = DatetimeOutputParser()

prompt = ChatPromptTemplate.from_template(
    """
    Give answer: When {person} was born?
    Format instruction: {format_instruction}
    """
)

In [ ]:
chain = prompt | llm | datetim_output_parser

In [ ]:
chain.invoke({"person": "Angus Young", "format_instruction": datetim_output_parser.get_format_instructions()})

#### Use 2 output parsers

In [ ]:
prompt = ChatPromptTemplate.from_template(
    """
    {question}
    {format_instruction}
    """
)

chain = prompt | llm | datetim_output_parser

In [ ]:
# Answer has wrong format
chain.invoke({"question": "When was openAI launched?", "format_instruction": datetim_output_parser.get_format_instructions()})

In [ ]:
from langchain.output_parsers import OutputFixingParser

fix_parser = OutputFixingParser.from_llm(parser=datetim_output_parser, llm=ChatOpenAI(model=LLM_MODEL))

chain = prompt | llm | (datetim_output_parser and fix_parser)

In [ ]:
chain.invoke({"question": "When was openAI launched?", "format_instruction": datetim_output_parser.get_format_instructions()})

#### Multiple chains

In [21]:
import warnings
warnings.filterwarnings("ignore")

In [15]:
from langchain.llms import HuggingFaceHub
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [157]:
prompt1 = ChatPromptTemplate.from_template("Write a huge story about {story_object}")
ll1 = ChatOpenAI(model=LLM_MODEL)

chain1 = prompt1 | ll1

In [158]:
prompt2 = ChatPromptTemplate.from_template(
    """
    You are pirate. Make a short review of provided story.
    Story: ```{story}```
    """
)
ll2 = ChatOpenAI(model=LLM_MODEL)
chain2 = {"story": chain1} | prompt2 | ll2

In [159]:
print(chain2.invoke({"story_object": "pizza"}).content)

Arr, me hearties! Gather 'round and listen to the tale of Giovanni, the pizzaiolo who turned pizza into a legend. This story be full of adventure and flavors that will make yer taste buds dance a jig!

Giovanni, a young and passionate pizzaiolo from a small village in Italy, inherited the art of pizza-making from his ancestors. His family's pizzeria, "La Dolce Pizza," became famous for its delicious and innovative creations. Giovanni's love for pizza knew no bounds, and he tirelessly perfected his techniques to create the ultimate pie.

One fateful day, a gust of wind blew in a group of travelers carrying exotic ingredients from a land called "Flavortopia." Intrigued, Giovanni invited them into his kitchen and learned about these mysterious flavors. With their guidance, Giovanni set sail on a grand adventure, leaving his pizzeria in the hands of his trusted assistant, Sofia.

In Flavortopia, Giovanni and Sofia discovered vibrant colors, tantalizing aromas, and the renowned chef, Nonna 

In [160]:
prompt3 = ChatPromptTemplate.from_template("```{text}```")
sumorization_model = HuggingFaceHub(repo_id="facebook/bart-large-cnn")
chain3 = {"text": chain2} | prompt3 | sumorization_model

In [163]:
print(chain3.invoke({"story_object": "pizza"}))

Cheesy Delights is a family-owned pizzeria in the city of Pizzaopolis. Greasy Bites, a large pizza chain, opens across the street. Nonna Maria and her family collaborate with the owners, sharin' their secret recipes and techniques. The partnership brings the best of both worlds to the city.
